# Parameter Selector
Implements bi-level optimisation model to calibrate a tradable performance standard to achieve environmental and economic objectives.

## Import packages

In [1]:
import os
import re
import time
import pickle
import itertools
from math import pi

import numpy as np
import pandas as pd

from pyomo.environ import *

import matplotlib.pyplot as plt
np.random.seed(10)

## Paths
Paths to relevant data and output directories.

In [2]:
class DirectoryPaths(object):
    "Paths to relevant directories"
    
    def __init__(self):
        self.data_dir = os.path.join(os.path.curdir, os.path.pardir, os.path.pardir, os.path.pardir, 'data')
        self.scenarios_dir = os.path.join(os.path.curdir, os.path.pardir, os.path.pardir, '1_create_scenarios')
        self.output_dir = os.path.join(os.path.curdir, 'output')

paths = DirectoryPaths()

## Model data
Import raw model data.

In [3]:
class RawData(object):
    "Collect input data"
    
    def __init__(self):
        
        # Paths to directories
        DirectoryPaths.__init__(self)
        
        
        # Network data
        # ------------
        # Nodes
        self.df_n = pd.read_csv(os.path.join(self.data_dir, 'egrimod-nem-dataset-v1.3', 'akxen-egrimod-nem-dataset-4806603', 'network', 'network_nodes.csv'), index_col='NODE_ID')

        # AC edges
        self.df_e = pd.read_csv(os.path.join(self.data_dir, 'egrimod-nem-dataset-v1.3', 'akxen-egrimod-nem-dataset-4806603', 'network', 'network_edges.csv'), index_col='LINE_ID')

        # HVDC links
        self.df_hvdc_links = pd.read_csv(os.path.join(self.data_dir, 'egrimod-nem-dataset-v1.3', 'akxen-egrimod-nem-dataset-4806603', 'network', 'network_hvdc_links.csv'), index_col='HVDC_LINK_ID')

        # AC interconnector links
        self.df_ac_i_links = pd.read_csv(os.path.join(self.data_dir, 'egrimod-nem-dataset-v1.3', 'akxen-egrimod-nem-dataset-4806603', 'network', 'network_ac_interconnector_links.csv'), index_col='INTERCONNECTOR_ID')

        # AC interconnector flow limits
        self.df_ac_i_limits = pd.read_csv(os.path.join(self.data_dir, 'egrimod-nem-dataset-v1.3', 'akxen-egrimod-nem-dataset-4806603', 'network', 'network_ac_interconnector_flow_limits.csv'), index_col='INTERCONNECTOR_ID')


        # Generators
        # ----------       
        # Generating unit information
        self.df_g = pd.read_csv(os.path.join(self.data_dir, 'egrimod-nem-dataset-v1.3', 'akxen-egrimod-nem-dataset-4806603', 'generators', 'generators.csv'), index_col='DUID', dtype={'NODE': int})
        self.df_g['SRMC_2016-17'] = self.df_g['SRMC_2016-17'].map(lambda x: x + np.random.uniform(0, 2))
        
               
        # Operating scenarios
        # -------------------
        with open(os.path.join(paths.scenarios_dir, 'output', '2_scenarios.pickle'), 'rb') as f:
            self.df_scenarios = pickle.load(f)

# Create object containing raw model data
raw_data = RawData() 

## Organise model data
Format and organise data.

In [4]:
class OrganiseData(object):
    "Organise data to be used in mathematical program"
    
    def __init__(self):
        # Load model data
        RawData.__init__(self)
        

    def get_admittance_matrix(self):
        "Construct admittance matrix for network"

        # Initialise dataframe
        df_Y = pd.DataFrame(data=0j, index=self.df_n.index, columns=self.df_n.index)

        # Off-diagonal elements
        for index, row in self.df_e.iterrows():
            fn, tn = row['FROM_NODE'], row['TO_NODE']
            df_Y.loc[fn, tn] += - (1 / (row['R_PU'] + 1j * row['X_PU'])) * row['NUM_LINES']
            df_Y.loc[tn, fn] += - (1 / (row['R_PU'] + 1j * row['X_PU'])) * row['NUM_LINES']

        # Diagonal elements
        for i in self.df_n.index:
            df_Y.loc[i, i] = - df_Y.loc[i, :].sum()

        # Add shunt susceptance to diagonal elements
        for index, row in self.df_e.iterrows():
            fn, tn = row['FROM_NODE'], row['TO_NODE']
            df_Y.loc[fn, fn] += (row['B_PU'] / 2) * row['NUM_LINES']
            df_Y.loc[tn, tn] += (row['B_PU'] / 2) * row['NUM_LINES']

        return df_Y
    
    
    def get_HVDC_incidence_matrix(self):
        "Incidence matrix for HVDC links"
        
        # Incidence matrix for HVDC links
        df = pd.DataFrame(index=self.df_n.index, columns=self.df_hvdc_links.index, data=0)

        for index, row in self.df_hvdc_links.iterrows():
            # From nodes assigned a value of 1
            df.loc[row['FROM_NODE'], index] = 1

            # To nodes assigned a value of -1
            df.loc[row['TO_NODE'], index] = -1
        
        return df
    
    
    def get_all_ac_edges(self):
        "Tuples defining from and to nodes for all AC edges (forward and reverse)"
        
        # Set of all AC edges
        edge_set = set()
        
        # Loop through edges, add forward and reverse direction indice tuples to set
        for index, row in model_data.df_e.iterrows():
            edge_set.add((row['FROM_NODE'], row['TO_NODE']))
            edge_set.add((row['TO_NODE'], row['FROM_NODE']))
        
        return edge_set
    
    def get_network_graph(self):
        "Graph containing connections between all network nodes"
        network_graph = {n: set() for n in model_data.df_n.index}

        for index, row in model_data.df_e.iterrows():
            network_graph[row['FROM_NODE']].add(row['TO_NODE'])
            network_graph[row['TO_NODE']].add(row['FROM_NODE'])
        
        return network_graph
    
    
    def get_all_dispatchable_fossil_generator_duids(self):
        "Fossil dispatch generator DUIDs"
        
        # Filter - keeping only fossil and scheduled generators
        mask = (model_data.df_g['FUEL_CAT'] == 'Fossil') & (model_data.df_g['SCHEDULE_TYPE'] == 'SCHEDULED')
        
        return model_data.df_g[mask].index    
    
    
    def get_intermittent_dispatch(self):
        "Dispatch from intermittent generators (solar, wind)"
        
        # Intermittent generator DUIDs
        intermittent_duids_mask = model_data.df_g['FUEL_CAT'].isin(['Wind', 'Solar'])
        intermittent_duids = model_data.df_g.loc[intermittent_duids_mask].index

        # Intermittent dispatch aggregated by node
        intermittent_dispatch =(model_data.df_dispatch.reindex(columns=intermittent_duids, fill_value=0)
                                .T
                                .join(model_data.df_g[['NODE']])
                                .groupby('NODE').sum()
                                .reindex(index=model_data.df_n.index, fill_value=0)
                                .T)
        
        # Make sure columns are of type datetime
        intermittent_dispatch.index = intermittent_dispatch.index.astype('datetime64[ns]')
        
        return intermittent_dispatch
    
    
    def get_hydro_dispatch(self):
        "Dispatch from hydro plant"
        
        # Dispatch from hydro plant
        hydro_duids_mask = self.df_g['FUEL_CAT'].isin(['Hydro'])
        hydro_duids = self.df_g.loc[hydro_duids_mask].index

        # Hydro plant dispatch aggregated by node
        hydro_dispatch = (self.df_dispatch.reindex(columns=hydro_duids, fill_value=0)
                          .T
                          .join(model_data.df_g[['NODE']])
                          .groupby('NODE').sum()
                          .reindex(index=self.df_n.index, fill_value=0)
                          .T)
        
        # Make sure columns are of type datetime
        hydro_dispatch.index = hydro_dispatch.index.astype('datetime64[ns]')
        
        return hydro_dispatch
    
    
    def get_reference_nodes(self):
        "Get reference node IDs"
        
        # Filter Regional Reference Nodes (RRNs) in Tasmania and Victoria.
        mask = (model_data.df_n['RRN'] == 1) & (model_data.df_n['NEM_REGION'].isin(['TAS1', 'VIC1']))
        reference_node_ids = model_data.df_n[mask].index
        
        return reference_node_ids
    
    
    def get_node_demand(self):   
        "Compute demand at each node for a given time period, t"

        def _node_demand(row):
            # NEM region for a given node
            region = row['NEM_REGION']

            # Load at node
            demand = self.df_load.loc[:, region] * row['PROP_REG_D']

            return demand
        node_demand = self.df_n.apply(_node_demand, axis=1).T
        
        return node_demand
    
    
    def get_generator_node_map(self, generators):
        "Get set of generators connected to each node"
        generator_node_map = (self.df_g.reindex(index=generators)
                              .reset_index()
                              .rename(columns={'OMEGA_G': 'DUID'})
                              .groupby('NODE').agg(lambda x: set(x))['DUID']
                              .reindex(self.df_n.index, fill_value=set()))
        
        return generator_node_map
    
    
    def get_ac_interconnector_branches(self):
        "Get all AC interconnector branches - check that flow directions for each branch are correct"

        # Check that from and to regions conform with regional power flow limit directions
        def check_flow_direction(row):
            if (row['FROM_REGION'] == self.df_ac_i_limits.loc[row.name, 'FROM_REGION']) & (row['TO_REGION'] == model_data.df_ac_i_limits.loc[row.name, 'TO_REGION']):
                return True
            else:
                return False
        # Flow directions are consistent between link and limit DataFrames if True
        flow_directions_conform = self.df_ac_i_links.apply(check_flow_direction, axis=1).all()
        if flow_directions_conform:
            print('Flow directions conform with regional flow limit directions: {0}'.format(flow_directions_conform))
        else:
            raise(Exception('Link flow directions inconsitent with regional flow forward limit definition'))

        # Forward links
        forward_links = self.df_ac_i_links.apply(lambda x: pd.Series({'INTERCONNECTOR_ID': '-'.join([x.name, 'FORWARD']), 'BRANCH': (x['FROM_NODE'], x['TO_NODE'])}), axis=1).set_index('INTERCONNECTOR_ID')
        
        # Reverse links
        reverse_links = self.df_ac_i_links.apply(lambda x: pd.Series({'INTERCONNECTOR_ID': '-'.join([x.name, 'REVERSE']), 'BRANCH': (x['TO_NODE'], x['FROM_NODE'])}), axis=1).set_index('INTERCONNECTOR_ID')
        
        # Combine forward and reverse links
        df = pd.concat([forward_links, reverse_links]).reset_index()
        
        # Construct branch ID
        df['BRANCH_ID'] = df.apply(lambda x: '_'.join(['L', str(x.name + 1)]), axis=1)
        df.set_index('BRANCH_ID', inplace=True)
        
        return df
    
    
    def get_ac_interconnector_flow_limits(self):
        "Get aggregate flow limits for each interconnector direction (both forward and reverse)"
        
        # Forward limits
        forward_limits = self.df_ac_i_limits.apply(lambda x: pd.Series({'LIMIT': x['FORWARD_LIMIT_MW'], 'INTERCONNECTOR_ID': '-'.join([x.name, 'FORWARD'])}), axis=1).set_index('INTERCONNECTOR_ID')

        # Reverse limits
        reverse_limits = self.df_ac_i_limits.apply(lambda x: pd.Series({'LIMIT': x['REVERSE_LIMIT_MW'], 'INTERCONNECTOR_ID': '-'.join([x.name, 'REVERSE'])}), axis=1).set_index('INTERCONNECTOR_ID')

        # Combine forward and reverse limits
        interconnector_limits = pd.concat([forward_limits, reverse_limits])
        
        return interconnector_limits
    
    
    def get_ac_interconnector_branch_ids(self):
        "Get branch IDs that consitute each interconnector"
        
        # Branch IDs for each interconnector
        df = self.get_ac_interconnector_branches().reset_index().groupby('INTERCONNECTOR_ID').apply(lambda x: list(x['BRANCH_ID']))
        
        return df
    
    
    def get_ac_interconnector_branch_node_incidence_matrix(self):
        "Incidence matrix showing if AC interconnector branch is defined as (+) or (-) flow for each node"
        
        # Branches constituting AC interconnectors
        interconnector_branches = self.get_ac_interconnector_branches()

        # Initialise interconnector branch - node incidence matrix
        df = pd.DataFrame(index=interconnector_branches.index, columns=self.df_n.index, data=0)

        for index, row in df.iterrows():
            # Branch from node
            from_node = interconnector_branches.loc[index, 'BRANCH'][0]

            # Branch to node
            to_node = interconnector_branches.loc[index, 'BRANCH'][1]

            # Update values in matrix
            df.loc[index, from_node] = 1
            df.loc[index, to_node] = -1

        return df.T

# Create object containing organised model data
model_data = OrganiseData()

## Model

In [5]:
def create_model(use_pu=None, variable_baseline=None, objective_type=None):
    """Create TPS baseline selection model
    
    Parameters
    ----------
    use_pu : bool
        Define if per-unit normalisation should be used. Re-scales parameters by system base power.
    
    variable_baseline : bool
        Specify if the baseline should be treated as a variable. E.g. find baseline that delivers given objective.
        
    objective_type : str
        Options:
            'feasibility' - find a feasible solution for given baseline
            'permit_price_target' - find baseline that delivers given permit price
            'weighted_rrn_price_target' - find baseline that targets weighted regional reference node (RRN) prices
            'nodal_electricity_price_target' - find baseline that targets nodal prices
            'minimise_electricity_price' - find baseline that minimises average wholesale electricity price
            
    Returns
    -------
    model : Pyomo model object
        Model object contains constraints and objectives corresponding to the given objective type
    """
    
    # Check parameters correctly specified
    if (use_pu is None) or (variable_baseline is None):
        raise(Exception('Must specify if baseline is variable, if per-unit system should be used, and type of objective for which model should be optimised'))
    
    
    # Mapping functions
    # -----------------
    def f_1(g):
        "Given generator, g, return the index of the node to which it is connected"
        return int(model_data.df_g.loc[g, 'NODE'])

    def f_2(h):
        "Given HVDC link, h, return the index of the link's 'from' node"
        return int(model_data.df_hvdc_links.loc[h, 'FROM_NODE'])

    def f_3(h):
        "Given HVDC link, h, return the index of the link's 'to' node"
        return int(model_data.df_hvdc_links.loc[h, 'TO_NODE'])
    
    def f_4(r):
        "Given NEM region, r, return index of region's Regional Reference Node (RRN)"
        return int(model_data.df_n[model_data.df_n['RRN'] == 1].reset_index().set_index('NEM_REGION').loc[r, 'NODE_ID'])


    # Construct model
    # ---------------
    # Initialise model
    model = ConcreteModel()


    # Sets
    # ----   
    # Nodes
    model.OMEGA_N = Set(initialize=model_data.df_n.index)

    # Generators
    model.OMEGA_G = Set(initialize=model_data.get_all_dispatchable_fossil_generator_duids())

    # AC edges
    ac_edges = model_data.get_all_ac_edges()
    model.OMEGA_NM = Set(initialize=ac_edges)

    # Sets of branches for which aggregate AC interconnector limits are defined
    ac_interconnector_flow_limits = model_data.get_ac_interconnector_flow_limits()
    model.OMEGA_J = Set(initialize=ac_interconnector_flow_limits.index)

    # HVDC links
    model.OMEGA_H = Set(initialize=model_data.df_hvdc_links.index)

    # Operating scenarios
    model.OMEGA_S = Set(initialize=model_data.df_scenarios.columns)
    
    # NEM regions
    model.OMEGA_R = Set(initialize=model_data.df_n['NEM_REGION'].unique())
    
    # Branches which constitute AC interconnectors in the network
    ac_interconnector_branch_node_incidence_matrix = model_data.get_ac_interconnector_branch_node_incidence_matrix()
    model.OMEGA_L = Set(initialize=ac_interconnector_branch_node_incidence_matrix.columns)


    # Maps
    # ----
    # Generator-node map
    generator_node_map = model_data.get_generator_node_map(model.OMEGA_G)

    # Network graph
    network_graph = model_data.get_network_graph()

    # Interconnectors and the branches to from which they are constituted
    ac_interconnector_branch_ids = model_data.get_ac_interconnector_branch_ids()
    
    # From and to nodes for each interconnector branch
    ac_interconnector_branches = model_data.get_ac_interconnector_branches()


    # Parameters
    # ----------
    # System base power
    model.BASE_POWER = Param(initialize=100)
    
    # Emissions intensity baseline (fixed)
    model.PHI = Param(initialize=0.95, mutable=True)

    # Admittance matrix
    admittance_matrix = model_data.get_admittance_matrix()
    def B_RULE(model, n, m):
        admittance_matrix_element = float(np.imag(admittance_matrix.loc[n, m]))
        if use_pu:
            return admittance_matrix_element
        else:
            return model.BASE_POWER * admittance_matrix_element
    model.B = Param(model.OMEGA_NM, rule=B_RULE)

    def P_H_MAX_RULE(s, h):
        forward_flow_limit = float(model_data.df_hvdc_links.loc[h, 'FORWARD_LIMIT_MW'])
        if use_pu:
            return forward_flow_limit / model.BASE_POWER
        else:
            return forward_flow_limit
    model.P_H_MAX = Param(model.OMEGA_H, rule=P_H_MAX_RULE)

    def P_H_MIN_RULE(s, h):
        reverse_flow_limit = float(model_data.df_hvdc_links.loc[h, 'REVERSE_LIMIT_MW'])
        if use_pu:
            return - reverse_flow_limit / model.BASE_POWER
        else:
            return - reverse_flow_limit
    model.P_H_MIN = Param(model.OMEGA_H, rule=P_H_MIN_RULE)

    # Reference nodes
    reference_nodes = model_data.get_reference_nodes()
    def S_R_RULE(model, n):
        if n in reference_nodes:
            return 1
        else:
            return 0
    model.S_R = Param(model.OMEGA_N, rule=S_R_RULE)
    
    # Maximum generator output
    def P_MAX_RULE(model, g):
        registered_capacity = float(model_data.df_g.loc[g, 'REG_CAP'])
        if use_pu:
            return registered_capacity / model.BASE_POWER
        else:
            return registered_capacity
    model.P_MAX = Param(model.OMEGA_G, rule=P_MAX_RULE)

    # Minimum generator output (set to 0)
    def P_MIN_RULE(model, g):
        minimum_output = 0
        if use_pu:
            return minimum_output / model.BASE_POWER
        else:
            return minimum_output
    model.P_MIN = Param(model.OMEGA_G, rule=P_MIN_RULE)

    # Generator short-run marginal costs
    def C_RULE(model, g):
        marginal_cost = float(model_data.df_g.loc[g, 'SRMC_2016-17'])
        if use_pu:
            return marginal_cost / model.BASE_POWER
        else:
            return marginal_cost
    model.C = Param(model.OMEGA_G, rule=C_RULE)

    # Generator emissions intensities
    def E_RULE(model, g):
        return float(model_data.df_g.loc[g, 'EMISSIONS'])
    model.E = Param(model.OMEGA_G, rule=E_RULE)

    # Max voltage angle difference between connected nodes
    model.THETA_DELTA = Param(initialize=float(pi / 2))

    # HVDC incidence matrix
    hvdc_incidence_matrix = model_data.get_HVDC_incidence_matrix()
    def K_RULE(model, n, h):
        return float(hvdc_incidence_matrix.loc[n, h])
    model.K = Param(model.OMEGA_N, model.OMEGA_H, rule=K_RULE)    

    # AC interconnector incidence matrix
    def S_L_RULE(model, n, l):
        return float(ac_interconnector_branch_node_incidence_matrix.loc[n, l])
    model.S_L = Param(model.OMEGA_N, model.OMEGA_L, rule=S_L_RULE)

    # Aggregate AC interconnector flow limits
    ac_interconnector_flow_limits = model_data.get_ac_interconnector_flow_limits()
    def F_RULE(model, j):
        power_flow_limit = float(ac_interconnector_flow_limits.loc[j, 'LIMIT'])
        if use_pu:
            return power_flow_limit / model.BASE_POWER
        else:
            return power_flow_limit
    model.F = Param(model.OMEGA_J, rule=F_RULE)

    # Big-M parameters
    def M_11_RULE(model, g):
        return model.P_MAX[g] - model.P_MIN[g]
    model.M_11 = Param(model.OMEGA_G, rule=M_11_RULE)

    def M_12_RULE(model, g):
        bound = 1e3
        if use_pu:
            return bound / model.BASE_POWER
        else:
            return bound
    model.M_12 = Param(model.OMEGA_G, rule=M_12_RULE)

    def M_21_RULE(model, g):
        return model.P_MAX[g] - model.P_MIN[g]
    model.M_21 = Param(model.OMEGA_G, rule=M_21_RULE)

    def M_22_RULE(model, g):
        bound = 1e3
        if use_pu:
            return bound / model.BASE_POWER
        else:
            return bound
    model.M_22 = Param(model.OMEGA_G, rule=M_22_RULE)

    def M_31_RULE(model, n, m):
        return float(pi)
    model.M_31 = Param(model.OMEGA_NM, rule=M_31_RULE)

    def M_32_RULE(model, n, m):
        bound = 1e3
        if use_pu:
            return bound / model.BASE_POWER
        else:
            return bound
    model.M_32 = Param(model.OMEGA_NM, rule=M_32_RULE)

    def M_41_RULE(model, j):
        if 'REVERSE' in j:
            new_index = j.replace('REVERSE', 'FORWARD')
        elif 'FORWARD' in j:
            new_index = j.replace('FORWARD', 'REVERSE')
        else:
            raise(Exception('REVERSE / FORWARD not in index name'))
        return model.F[j] + model.F[new_index]
    model.M_41 = Param(model.OMEGA_J, rule=M_41_RULE)

    def M_42_RULE(model, j):
        bound = 1e3
        if use_pu:
            return bound / model.BASE_POWER
        else:
            return bound
    model.M_42 = Param(model.OMEGA_J, rule=M_42_RULE)

    def M_51_RULE(model, h):
        return model.P_H_MAX[h] - model.P_H_MIN[h]
    model.M_51 = Param(model.OMEGA_H, rule=M_51_RULE)

    def M_52_RULE(model, h):
        bound = 1e3
        if use_pu:
            return bound / model.BASE_POWER
        else:
            return bound
    model.M_52 = Param(model.OMEGA_H, rule=M_52_RULE)

    def M_61_RULE(model, h):
        return model.P_H_MAX[h] - model.P_H_MIN[h]
    model.M_61 = Param(model.OMEGA_H, rule=M_61_RULE)

    def M_62_RULE(model, h):
        bound = 1e3
        if use_pu:
            return bound / model.BASE_POWER
        else:
            return bound
    model.M_62 = Param(model.OMEGA_H, rule=M_62_RULE)

    def M_71_RULE(model):
        bound = 1e4
        if use_pu:
            return 1e4 / model.BASE_POWER
        else:
            return bound
    model.M_71 = Param(rule=M_71_RULE)

    def M_72_RULE(model):
        bound = 1e4
        if use_pu:
            return bound / model.BASE_POWER
        else:
            return bound
    model.M_72 = Param(rule=M_72_RULE)


    # Variables
    # ---------
    # Permit market constraint dual variable
    model.tau = Var()

    # Permit market constraint binary variable
    model.GAMMA_7 = Var(within=Binary)

    # Additional sets, parameters, variable, and constraints if baseline is variable
    if variable_baseline:
        # Largest integer for baseline selection parameter index
        model.U = 10

        # Set of integers used to select discretized baseline
        model.OMEGA_U = Set(initialize=range(0, model.U + 1))

        # Minimum emissions intensity baseline
        def PHI_MIN_RULE(model):
            return float(0.8)
        model.PHI_MIN = Param(rule=PHI_MIN_RULE)

        # Maximum emissions intensity baseline
        def PHI_MAX_RULE(model):
            return float(1.3)
        model.PHI_MAX = Param(rule=PHI_MAX_RULE)

        # Emissions intensity baseline increment
        model.PHI_DELTA = Param(initialize=float((model.PHI_MAX - model.PHI_MIN) / (2**model.U)))

        # Parameter equal 2^u used when selecting discrete emissions intensity baseline
        def TWO_U_RULE(model, u):
            return float(2**u)
        model.TWO_U = Param(model.OMEGA_U, rule=TWO_U_RULE)

        # Binary variables used to determine discretized emissions intensity baseline choice
        model.PSI = Var(model.OMEGA_U, within=Binary)

        # Composite variable - PSI x tau - used to linearise bi-linear term
        model.z_1 = Var(model.OMEGA_U)

        # Big-M parameter used used to make z_1=0 when PSI=0, and z_1=tau when PSI=1
        def L_1_RULE(model):
            bound = 1e3
            if use_pu:
                return bound / model.BASE_POWER
            else:
                return bound
        model.L_1 = Param(rule=L_1_RULE)

        # Big-M paramter used to make z_2=0 when PSI=0, and z_2=p when PSI=1
        def L_2_RULE(model, g):
            return model.P_MAX[g] - model.P_MIN[g]
        model.L_2 = Param(model.OMEGA_G, rule=L_2_RULE)

        # Constraints such that z_1=0 when PSI=0 and z_1 = tau when PSI=1
        def Z_1_CONSTRAINT_1_RULE(model, u):
            return 0 <= model.tau - model.z_1[u]
        model.Z_1_CONSTRAINT_1 = Constraint(model.OMEGA_U, rule=Z_1_CONSTRAINT_1_RULE)

        def Z_1_CONSTRAINT_2_RULE(model, u):
            return model.tau - model.z_1[u] <= model.L_1 * (1 - model.PSI[u])
        model.Z_1_CONSTRAINT_2 = Constraint(model.OMEGA_U, rule=Z_1_CONSTRAINT_2_RULE)

        def Z_1_CONSTRAINT_3_RULE(model, u):
            return 0 <= model.z_1[u]
        model.Z_1_CONSTRAINT_3 = Constraint(model.OMEGA_U, rule=Z_1_CONSTRAINT_3_RULE)

        def Z_1_CONSTRAINT_4_RULE(model, u):
            return model.z_1[u] <= model.L_1 * model.PSI[u]
        model.Z_1_CONSTRAINT_4 = Constraint(model.OMEGA_U, rule=Z_1_CONSTRAINT_4_RULE)

        # Discretised emissions intensity baseline value
        model.PHI_DISCRETE = Expression(expr=model.PHI_MIN + (model.PHI_DELTA * sum(model.TWO_U[u] * model.PSI[u] for u in model.OMEGA_U)))


    def SCENARIO_RULE(b, s):
        "Block of constraints describing optimality conditions for each scenario"

        # Parameters
        # ----------       
        # Fixed power injections
        def R_RULE(b, n):
            fixed_injection = float(model_data.df_scenarios.loc[('intermittent', n), s] + model_data.df_scenarios.loc[('hydro', n), s])
            
            # Remove very small fixed power injections to improve numerical conditioning
            if fixed_injection < 1:
                fixed_injection = 0
            
            if use_pu:
                return fixed_injection / model.BASE_POWER
            else:
                return fixed_injection
        b.R = Param(model.OMEGA_N, rule=R_RULE)

        # Demand
        def D_RULE(b, n):
            demand = float(model_data.df_scenarios.loc[('demand', n), s])
            
            # Remove small demand to improve numerical conditioning
            if demand < 1:
                demand = 0
                        
            if use_pu:
                return demand / model.BASE_POWER
            else:
                return demand
        b.D = Param(model.OMEGA_N, rule=D_RULE)
        
        # Proportion of total demand consumed in each region
        def ZETA_RULE(b, r):            
            # Region demand
            region_demand = float((model_data.df_scenarios
                                   .join(model_data.df_n[['NEM_REGION']], how='left')
                                   .reset_index()
                                   .groupby(['NEM_REGION','level'])
                                   .sum()
                                   .loc[(r, 'demand'), s]))

            # Total demand
            total_demand = float(model_data.df_scenarios.reset_index().groupby('level').sum().loc['demand', s])
            
            # Proportion of demand consumed in region
            demand_proportion = float(region_demand / total_demand)
            
            return demand_proportion
        b.ZETA = Param(model.OMEGA_R, rule=ZETA_RULE)           

        # Scenario duration
        def RHO_RULE(b):
            return float(model_data.df_scenarios.loc[('hours', 'duration'), s] / 8760)
        b.RHO = Param(rule=RHO_RULE)


        # Primal variables
        # ----------------
        # Generator output
        b.p = Var(model.OMEGA_G)

        # HVDC link flow
        b.p_H = Var(model.OMEGA_H)

        # Node voltage angle
        b.theta = Var(model.OMEGA_N)


        # Dual variables
        # --------------
        # Min power output constraint dual varaible
        b.mu_1 = Var(model.OMEGA_G)

        # Max power output constraint dual variable
        b.mu_2 = Var(model.OMEGA_G)

        # Max voltage angle difference constraint dual variable
        b.mu_3 = Var(model.OMEGA_NM)

        # AC link power flow constraint dual variable
        b.mu_4 = Var(model.OMEGA_J)

        # Min HVDC flow constraint dual variable
        b.mu_5 = Var(model.OMEGA_H)

        # Max HVDC flow constraint dual variable
        b.mu_6 = Var(model.OMEGA_H)

        # Reference node voltage angle constraint dual variable
        b.nu_1 = Var(model.OMEGA_N)

        # Node power balance constraint dual variable
        b.lamb = Var(model.OMEGA_N)


        # Binary variables
        # ----------------
        # Min power output binary variable
        b.GAMMA_1 = Var(model.OMEGA_G, within=Binary)

        # Max power output binary variable
        b.GAMMA_2 = Var(model.OMEGA_G, within=Binary)

        # Max voltage angle difference binary variable
        b.GAMMA_3 = Var(model.OMEGA_NM, within=Binary)

        # AC link power flow dual variable
        b.GAMMA_4 = Var(model.OMEGA_J, within=Binary)

        # Min HVDC flow binary variable
        b.GAMMA_5 = Var(model.OMEGA_H, within=Binary)

        # Max HVDC flow binary variable
        b.GAMMA_6 = Var(model.OMEGA_H, within=Binary)

        # Parameters and variables if emissions intensity baseline is variable
        if variable_baseline:
            # Composite variable - PSI x p - used to linearise bi-linear term
            b.z_2 = Var(model.OMEGA_U * model.OMEGA_G)

            # Constraints such that z_2=0 when PSI=0, and z_2=p when PSI=1
            def Z_2_CONSTRAINT_1_RULE(b, u, g):
                return 0 <= b.p[g] - b.z_2[u, g]
            b.Z_2_CONSTRAINT_1 = Constraint(model.OMEGA_U * model.OMEGA_G, rule=Z_2_CONSTRAINT_1_RULE)

            def Z_2_CONSTRAINT_2_RULE(b, u, g):
                return b.p[g] - b.z_2[u, g] <= model.L_2[g] * (1 - model.PSI[u])
            b.Z_2_CONSTRAINT_2 = Constraint(model.OMEGA_U * model.OMEGA_G, rule=Z_2_CONSTRAINT_2_RULE)

            def Z_2_CONSTRAINT_3_RULE(b, u, g):
                return 0 <= b.z_2[u, g]
            b.Z_2_CONSTRAINT_3 = Constraint(model.OMEGA_U * model.OMEGA_G, rule=Z_2_CONSTRAINT_3_RULE)

            def Z_2_CONSTRAINT_4_RULE(b, u, g):
                return b.z_2[u, g] <= model.L_2[g] * model.PSI[u]
            b.Z_2_CONSTRAINT_4 = Constraint(model.OMEGA_U * model.OMEGA_G, rule=Z_2_CONSTRAINT_4_RULE)


        # Constraints
        # -----------
        # First order conditions
        # If baseline is fixed
        def FOC_1_RULE(b, g):
            return (model.C[g] 
                    + ((model.E[g] - model.PHI) * model.tau) 
                    - b.mu_1[g] 
                    + b.mu_2[g] 
                    - b.lamb[f_1(g)] == 0)

        # Linearised first order condition if baseline is variable
        def FOC_1_LIN_RULE(b, g):
            return (model.C[g] 
                    + ((model.E[g] - model.PHI_MIN) * model.tau) 
                    - (model.PHI_DELTA * sum(model.TWO_U[u] * model.z_1[u] for u in model.OMEGA_U)) 
                    - b.mu_1[g] + b.mu_2[g] - b.lamb[f_1(g)] == 0)

        # Activate appropriate constraint depending on whether baseline is fixed or variable
        if variable_baseline:
            # Activate if variable
            b.FOC_1_LIN = Constraint(model.OMEGA_G, rule=FOC_1_LIN_RULE)
        else:
            # Activate if fixed
            b.FOC_1 = Constraint(model.OMEGA_G, rule=FOC_1_RULE)

        def FOC_2_RULE(b, n):
            return (sum(b.mu_3[n, m] 
                        - b.mu_3[m, n] 
                        + (b.lamb[n] * model.B[n, m]) 
                        - (b.lamb[m] * model.B[m, n]) for m in network_graph[n]) 
                    + (b.nu_1[n] * model.S_R[n]) 
                    + sum(model.B[ac_interconnector_branches.loc[l, 'BRANCH']] * b.mu_4[j] * model.S_L[n, l]
                          for j in model.OMEGA_J for l in ac_interconnector_branch_ids.loc[j]
                         ) == 0)
        b.FOC_2 = Constraint(model.OMEGA_N, rule=FOC_2_RULE)

        def FOC_3_RULE(b, h):
            return ((model.K[f_2(h), h] * b.lamb[f_2(h)]) 
                    + (model.K[f_3(h), h] * b.lamb[f_3(h)]) 
                    - b.mu_5[h] 
                    + b.mu_6[h] == 0)
        b.FOC_3 = Constraint(model.OMEGA_H, rule=FOC_3_RULE)

        def EQUALITY_CONSTRAINT_1_RULE(b, n):
            if model.S_R[n] == 1:
                return b.theta[n] == 0
            else:
                return Constraint.Skip
        b.EQUALITY_CONSTRAINT_1 = Constraint(model.OMEGA_N, rule=EQUALITY_CONSTRAINT_1_RULE)

        def EQUALITY_CONSTRAINT_2_RULE(b, n):
            return (b.D[n] 
                    - b.R[n] 
                    - sum(b.p[g] for g in generator_node_map[n]) 
                    + sum(model.B[n, m] * (b.theta[n] - b.theta[m]) for m in network_graph[n]) 
                    + sum(model.K[n, h] * b.p_H[h] for h in model.OMEGA_H) == 0)
        b.EQUALITY_CONSTRAINT_2 = Constraint(model.OMEGA_N, rule=EQUALITY_CONSTRAINT_2_RULE)

        # Linearised complementarity constraints
        # --------------------------------------
        # Min power output
        def LIN_COMP_1_1_RULE(b, g):
            return model.P_MIN[g] - b.p[g] <= 0
        b.LIN_COMP_1_1 = Constraint(model.OMEGA_G, rule=LIN_COMP_1_1_RULE)

        def LIN_COMP_1_2_RULE(b, g):
            return b.mu_1[g] >= 0
        b.LIN_COMP_1_2 = Constraint(model.OMEGA_G, rule=LIN_COMP_1_2_RULE)

        def LIN_COMP_1_3_RULE(b, g):
            return b.p[g] - model.P_MIN[g] <= b.GAMMA_1[g] * model.M_11[g]
        b.LIN_COMP_1_3 = Constraint(model.OMEGA_G, rule=LIN_COMP_1_3_RULE)

        def LIN_COMP_1_4_RULE(b, g):
            return b.mu_1[g] <= (1 - b.GAMMA_1[g]) * model.M_12[g]
        b.LIN_COMP_1_4 = Constraint(model.OMEGA_G, rule=LIN_COMP_1_4_RULE)

        # Max power output
        def LIN_COMP_2_1_RULE(b, g):
            return b.p[g] - model.P_MAX[g] <= 0
        b.LIN_COMP_2_1 = Constraint(model.OMEGA_G, rule=LIN_COMP_2_1_RULE)

        def LIN_COMP_2_2_RULE(b, g):
            return b.mu_2[g] >= 0
        b.LIN_COMP_2_2 = Constraint(model.OMEGA_G, rule=LIN_COMP_2_2_RULE)

        def LIN_COMP_2_3_RULE(b, g):
            return model.P_MAX[g] - b.p[g] <= b.GAMMA_2[g] * model.M_21[g]
        b.LIN_COMP_2_3 = Constraint(model.OMEGA_G, rule=LIN_COMP_2_3_RULE)

        def LIN_COMP_2_4_RULE(b, g):
            return b.mu_2[g] <= (1 - b.GAMMA_2[g]) * model.M_22[g]
        b.LIN_COMP_2_4 = Constraint(model.OMEGA_G, rule=LIN_COMP_2_4_RULE)

        # Max voltage angle difference between connected nodes
        def LIN_COMP_3_1_RULE(b, n, m):
            return b.theta[n] - b.theta[m] - model.THETA_DELTA <= 0
        b.LIN_COMP_3_1 = Constraint(model.OMEGA_NM, rule=LIN_COMP_3_1_RULE)

        def LIN_COMP_3_2_RULE(b, n, m):
            return b.mu_3[n, m] >= 0
        b.LIN_COMP_3_2 = Constraint(model.OMEGA_NM, rule=LIN_COMP_3_2_RULE)

        def LIN_COMP_3_3_RULE(b, n, m):
            return model.THETA_DELTA + b.theta[m] - b.theta[n] <= b.GAMMA_3[n, m] * model.M_31[n, m]
        b.LIN_COMP_3_3 = Constraint(model.OMEGA_NM, rule=LIN_COMP_3_3_RULE)

        def LIN_COMP_3_4_RULE(b, n, m):
            return b.mu_3[n, m] <= (1 - b.GAMMA_3[n, m]) * model.M_32[n, m]
        b.LIN_COMP_3_4 = Constraint(model.OMEGA_NM, rule=LIN_COMP_3_4_RULE)

        # Interconnector flow limits
        def LIN_COMP_4_1_RULE(b, j):
            branches = [ac_interconnector_branches.loc[branch_id, 'BRANCH'] for branch_id in ac_interconnector_branch_ids.loc[j]]
            return sum(model.B[n, m] * (b.theta[n] - b.theta[m]) for n, m in branches) - model.F[j] <= 0
        b.LIN_COMP_4_1 = Constraint(model.OMEGA_J, rule=LIN_COMP_4_1_RULE)

        def LIN_COMP_4_2_RULE(b, j):
            return b.mu_4[j] >= 0
        b.LIN_COMP_4_2 = Constraint(model.OMEGA_J, rule=LIN_COMP_4_2_RULE)

        def LIN_COMP_4_3_RULE(b, j):
            branches = [ac_interconnector_branches.loc[branch_id, 'BRANCH'] for branch_id in ac_interconnector_branch_ids.loc[j]]
            return model.F[j] - sum(model.B[n, m] * (b.theta[n] - b.theta[m]) for n, m in branches) <= b.GAMMA_4[j] * model.M_41[j]
        b.LIN_COMP_4_3 = Constraint(model.OMEGA_J, rule=LIN_COMP_4_3_RULE)

        def LIN_COMP_4_4_RULE(b, j):
            return b.mu_4[j] <= (1 - b.GAMMA_4[j]) * model.M_42[j]
        b.LIN_COMP_4_4 = Constraint(model.OMEGA_J, rule=LIN_COMP_4_4_RULE)

        # HVDC link flow limits
        def LIN_COMP_5_1_RULE(b, h):
            return model.P_H_MIN[h] - b.p_H[h] <= 0
        b.LIN_COMP_5_1 = Constraint(model.OMEGA_H, rule=LIN_COMP_5_1_RULE)

        def LIN_COMP_5_2_RULE(b, h):
            return b.mu_5[h] >= 0
        b.LIN_COMP_5_2 = Constraint(model.OMEGA_H, rule=LIN_COMP_5_2_RULE)

        def LIN_COMP_5_3_RULE(b, h):
            return b.p_H[h] - model.P_H_MIN[h] <= b.GAMMA_5[h] * model.M_51[h]
        b.LIN_COMP_5_3 = Constraint(model.OMEGA_H, rule=LIN_COMP_5_3_RULE)

        def LIN_COMP_5_4_RULE(b, h):
            return b.mu_5[h] <= (1 - b.GAMMA_5[h]) * model.M_52[h]
        b.LIN_COMP_5_4 = Constraint(model.OMEGA_H, rule=LIN_COMP_5_4_RULE)

        def LIN_COMP_6_1_RULE(b, h):
            return b.p_H[h] - model.P_H_MAX[h] <= 0
        b.LIN_COMP_6_1 = Constraint(model.OMEGA_H, rule=LIN_COMP_6_1_RULE)

        def LIN_COMP_6_2_RULE(b, h):
            return b.mu_6[h] >= 0
        b.LIN_COMP_6_2 = Constraint(model.OMEGA_H, rule=LIN_COMP_6_2_RULE)

        def LIN_COMP_6_3_RULE(b, h):
            return model.P_H_MAX[h] - b.p_H[h] <= b.GAMMA_6[h] * model.M_61[h]
        b.LIN_COMP_6_3 = Constraint(model.OMEGA_H, rule=LIN_COMP_6_3_RULE)

        def LIN_COMP_6_4_RULE(b, h):
            return b.mu_6[h] <= (1 - b.GAMMA_6[h]) * model.M_62[h]
        b.LIN_COMP_6_4 = Constraint(model.OMEGA_H, rule=LIN_COMP_6_4_RULE)
    model.SCENARIO = Block(model.OMEGA_S, rule=SCENARIO_RULE)

    # Permit market complementarity constraints
    def PERMIT_MARKET_1_RULE(model):
        return sum(model.SCENARIO[s].RHO * ((model.E[g] - model.PHI) * model.SCENARIO[s].p[g]) for g in model.OMEGA_G for s in model.OMEGA_S) <= 0

    def PERMIT_MARKET_1_LIN_RULE(model):
        return (sum(model.SCENARIO[s].RHO * (((model.E[g] - model.PHI_MIN) * model.SCENARIO[s].p[g]) 
                                            - (model.PHI_DELTA * sum(model.TWO_U[u] * model.SCENARIO[s].z_2[u, g] for u in model.OMEGA_U))) for g in model.OMEGA_G for s in model.OMEGA_S) <= 0)

    def PERMIT_MARKET_2_RULE(model):
        return model.tau >= 0
    model.PERMIT_MARKET_2 = Constraint(rule=PERMIT_MARKET_2_RULE)

    def PERMIT_MARKET_3_RULE(model):
        return sum(model.SCENARIO[s].RHO * ((model.PHI - model.E[g]) * model.SCENARIO[s].p[g]) for g in model.OMEGA_G for s in model.OMEGA_S) <= model.GAMMA_7 * model.M_71

    def PERMIT_MARKET_3_LIN_RULE(model):
        return (sum(model.SCENARIO[s].RHO * (((model.PHI_MIN - model.E[g]) * model.SCENARIO[s].p[g]) 
                                            + (model.PHI_DELTA * sum(model.TWO_U[u] * model.SCENARIO[s].z_2[u, g] for u in model.OMEGA_U))) for g in model.OMEGA_G for s in model.OMEGA_S) <= model.GAMMA_7 * model.M_71)

    def PERMIT_MARKET_4_RULE(model):
        return model.tau <= (1 - model.GAMMA_7) * model.M_72
    model.PERMIT_MARKET_4 = Constraint(rule=PERMIT_MARKET_4_RULE)

    # Use appropriate constraint formulations depending on whether emissions intensity baseline is variable or not
    if variable_baseline:
        model.PERMIT_MARKET_1_LIN = Constraint(rule=PERMIT_MARKET_1_LIN_RULE)
        model.PERMIT_MARKET_3_LIN = Constraint(rule=PERMIT_MARKET_3_LIN_RULE)
    else:
        model.PERMIT_MARKET_1 = Constraint(rule=PERMIT_MARKET_1_RULE)
        model.PERMIT_MARKET_3 = Constraint(rule=PERMIT_MARKET_3_RULE)


    # Expressions
    # -----------
    # Total revenue from electricity sales
    model.TOTAL_REVENUE = Expression(expr=sum(model.SCENARIO[s].RHO * model.SCENARIO[s].lamb[n] * model.SCENARIO[s].D[n] for s in model.OMEGA_S for n in model.OMEGA_N))

    # Total demand
    model.TOTAL_DEMAND = Expression(expr=sum(model.SCENARIO[s].RHO * model.SCENARIO[s].D[n] for s in model.OMEGA_S for n in model.OMEGA_N))

    # Average price
    model.AVERAGE_ELECTRICITY_PRICE = Expression(expr=model.TOTAL_REVENUE / model.TOTAL_DEMAND)

    # Weighted RRN price
    model.WEIGHTED_RRN_PRICE = Expression(expr=sum(model.SCENARIO[s].RHO * model.SCENARIO[s].ZETA[r] * model.SCENARIO[s].lamb[f_4(r)] for s in model.OMEGA_S for r in model.OMEGA_R))


    # Objective functions
    # -------------------
    # Feasibility
    if objective_type == 'feasibility':
        model.dummy = Var(bounds=(0, 1))
        model.OBJECTIVE = Objective(expr=model.dummy, sense=minimize)
        
        
    # Permit price target
    elif objective_type == 'permit_price_target':
        
        def PERMIT_PRICE_TARGET_RULE(model):
            # Permit price target [$/tCO2]
            permit_price_target = 30
            
            if use_pu:
                return permit_price_target / model.BASE_POWER
            else:
                return permit_price_target
        model.PERMIT_PRICE_TARGET = Param(initialize=PERMIT_PRICE_TARGET_RULE, mutable=True)
        
        # Dummy variables used to target given permit price
        model.PERMIT_PRICE_TARGET_X_1 = Var(within=NonNegativeReals)
        model.PERMIT_PRICE_TARGET_X_2 = Var(within=NonNegativeReals)
        
        # Constraints to minimise difference between permit price and target
        model.PERMIT_PRICE_TARGET_CONSTRAINT_1 = Constraint(expr=model.PERMIT_PRICE_TARGET_X_1 >= model.PERMIT_PRICE_TARGET - model.tau)
        model.PERMIT_PRICE_TARGET_CONSTRAINT_2 = Constraint(expr=model.PERMIT_PRICE_TARGET_X_2 >= model.tau - model.PERMIT_PRICE_TARGET)
        
        # Objective function
        model.OBJECTIVE = Objective(expr=model.PERMIT_PRICE_TARGET_X_1 + model.PERMIT_PRICE_TARGET_X_2)
        
        
    # Weighted RRN price target        
    elif objective_type == 'weighted_rrn_price_target':
        # Weighted RRN price target
        def WEIGHTED_RRN_PRICE_TARGET_RULE(model):
            # Price target [$/MWh]
            weighted_rrn_price_target = 36

            if use_pu:
                return weighted_rrn_price_target / model.BASE_POWER
            else:
                return weighted_rrn_price_target
        model.WEIGHTED_RRN_PRICE_TARGET = Param(initialize=WEIGHTED_RRN_PRICE_TARGET_RULE, mutable=True)
        
        # Dummy variables used to minimise difference between average price and price target
        model.WEIGHTED_RRN_PRICE_X_1 = Var(within=NonNegativeReals)
        model.WEIGHTED_RRN_PRICE_X_2 = Var(within=NonNegativeReals)
        
        # Constraints used to minimise difference between RRN price target and RRN price
        model.WEIGHTED_RRN_PRICE_TARGET_CONSTRAINT_1 = Constraint(expr=model.WEIGHTED_RRN_PRICE_X_1 >= model.WEIGHTED_RRN_PRICE - model.WEIGHTED_RRN_PRICE_TARGET)
        model.WEIGHTED_RRN_PRICE_TARGET_CONSTRAINT_2 = Constraint(expr=model.WEIGHTED_RRN_PRICE_X_2 >= model.WEIGHTED_RRN_PRICE_TARGET - model.WEIGHTED_RRN_PRICE)
        
        # Weighted RRN price targeting objective function
        model.OBJECTIVE = Objective(expr=model.WEIGHTED_RRN_PRICE_X_1 + model.WEIGHTED_RRN_PRICE_X_2)
        
        
    # Nodal electricity price target
    elif objective_type == 'nodal_electricity_price_target':
        def NODAL_ELECTRICITY_PRICE_TARGET_RULE(model, s, n):
            # Price target [$/MWh]
            target_nodal_price = 30
            
            if use_pu:
                return target_nodal_price / model.BASE_POWER
            else:
                return target_nodal_price
        model.NODAL_ELECTRICITY_PRICE_TARGET = Param(model.OMEGA_S, model.OMEGA_N, initialize=NODAL_ELECTRICITY_PRICE_TARGET_RULE, mutable=True)
        
        # Dummy variables used to minimise difference between nodal price and target
        model.NODAL_ELECTRICITY_PRICE_X_1 = Var(model.OMEGA_S, model.OMEGA_N, within=NonNegativeReals)
        model.NODAL_ELECTRICITY_PRICE_X_2 = Var(model.OMEGA_S, model.OMEGA_N, within=NonNegativeReals)
        
        # Constraints to minimise difference between nodal price and target
        def NODAL_ELECTRICTY_PRICE_TARGET_CONSTRAINT_1_RULE(model, s, n):
            return model.NODAL_ELECTRICITY_PRICE_X_1[s, n] >= model.SCENARIO[s].lamb[n] - model.NODAL_ELECTRICITY_PRICE_TARGET[s, n]
        model.NODAL_ELECTRICTY_PRICE_TARGET_CONSTRAINT_1 = Constraint(model.OMEGA_S, model.OMEGA_N, rule=NODAL_ELECTRICTY_PRICE_TARGET_CONSTRAINT_1_RULE)

        def NODAL_ELECTRICTY_PRICE_TARGET_CONSTRAINT_2_RULE(model, s, n):
            return model.NODAL_ELECTRICITY_PRICE_X_2[s, n] >= model.NODAL_ELECTRICITY_PRICE_TARGET[s, n] -  model.SCENARIO[s].lamb[n]
        model.NODAL_ELECTRICTY_PRICE_TARGET_CONSTRAINT_2 = Constraint(model.OMEGA_S, model.OMEGA_N, rule=NODAL_ELECTRICTY_PRICE_TARGET_CONSTRAINT_2_RULE)

        # Nodal price objective
        model.OBJECTIVE = Objective(expr=sum(model.SCENARIO[s].RHO * model.SCENARIO[s].D[n] * (model.NODAL_ELECTRICITY_PRICE_X_1[s, n] + model.NODAL_ELECTRICITY_PRICE_X_2[s, n]) for s in model.OMEGA_S for n in model.OMEGA_N))
        
        
    # Minimise average electricity price  
    elif objective_type == 'minimise_electricity_price':
        model.OBJECTIVE = Objective(expr=model.AVERAGE_ELECTRICITY_PRICE, sense=minimize)
        
        # Add constraint to put a lower-bound on the average electricity price (help solver)
        model.AVERAGE_PRICE_MIN = Param(initialize=0.3, mutable=True)
        model.AVERAGE_PRICE_LOWER_BOUND = Constraint(expr=model.AVERAGE_ELECTRICITY_PRICE >= model.AVERAGE_PRICE_MIN)

    else:
        raise(Exception('Invalid objective type'))

    return model

Setup solver.

In [6]:
# Setup solver
# ------------
solver = 'cplex'
solver_io = 'mps'
opt = SolverFactory(solver, solver_io=solver_io)

Solve model for different emissions intensity baselines.

In [7]:
def run_emissions_intensity_baseline_scenarios():
    "Run model for different emissions intensity baseline scenarios"
    
    # Instantiate model object
    model = create_model(use_pu=True, variable_baseline=False, objective_type='feasibility')
    opt.options['mip tolerances absmipgap'] = 1e-6
    opt.options['emphasis mip'] = 1 # Emphasise feasibility

    # Solve model for different PHI scenarios
    for baseline in np.linspace(1.1, 0.9, 41):
        # Dictionary in which to store results
        fixed_baseline_results = dict()

        # Update baseline
        print('Emissions intensity baseline: {0} tCO2/MWh'.format(baseline))
        model.PHI = baseline

        # Model results
        res = opt.solve(model, keepfiles=False, tee=True, warmstart=True)
        model.solutions.store_to(res)

        # Place results in DataFrame
        try:
            df = pd.DataFrame(res['Solution'][0])
            fixed_baseline_results = {'FIXED_BASELINE': model.PHI.value, 'results': df}
        except:
            fixed_baseline_results = {'FIXED_BASELINE': model.PHI.value, 'results': 'infeasible'}
            print('Baseline {0} is infeasible'.format(model.PHI.value))   

        # Try to print results
        try:
            print(model.AVERAGE_ELECTRICITY_PRICE.display())
        except:
            pass

        try:
            print(model.tau.display())
        except:
            pass

        # Save results
        filename = 'fixed_baseline_results_phi_{0:.3f}.pickle'.format(model.PHI.value)
        with open(os.path.join(paths.output_dir, filename), 'wb') as f:
            pickle.dump(fixed_baseline_results, f)
            
run_emissions_intensity_baseline_scenarios()

Flow directions conform with regional flow limit directions: True
Flow directions conform with regional flow limit directions: True
Flow directions conform with regional flow limit directions: True
Emissions intensity baseline: 1.1 tCO2/MWh

Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 12.10.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2019.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile '/home/compo/Desktop/git/research/phd-thesis/repos/tps-parameterisation/src/2_parameter_selector/kkt-approach/tmpvhzfahl0.cplex.log' open.
CPLEX> New value for absolute mixed integer optimality gap tolerance: 1e-06
CPLEX> New value for emphasis for MIP optimization: 1
CPLEX> Specified objective sense: MINIMIZE
Selected objective  name:  x16566
Selected RHS 

CPLEX> AVERAGE_ELECTRICITY_PRICE : Size=1
    Key  : Value
    None : 0.26059764018742865
None
tau : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :  None :   0.0 :  None : False : False :  Reals
None
Emissions intensity baseline: 1.0950000000000002 tCO2/MWh

Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 12.10.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2019.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile '/home/compo/Desktop/git/research/phd-thesis/repos/tps-parameterisation/src/2_parameter_selector/kkt-approach/tmpgvkn18c6.cplex.log' open.
CPLEX> New value for absolute mixed integer optimality gap tolerance: 1e-06
CPLEX> New value for emphasis for MIP optimization: 1
CPLEX> Specified objective sense

CPLEX> AVERAGE_ELECTRICITY_PRICE : Size=1
    Key  : Value
    None : 0.2605976401867579
None
tau : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :  None :   0.0 :  None : False : False :  Reals
None
Emissions intensity baseline: 1.0850000000000002 tCO2/MWh

Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 12.10.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2019.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile '/home/compo/Desktop/git/research/phd-thesis/repos/tps-parameterisation/src/2_parameter_selector/kkt-approach/tmppw29_0dn.cplex.log' open.
CPLEX> New value for absolute mixed integer optimality gap tolerance: 1e-06
CPLEX> New value for emphasis for MIP optimization: 1
CPLEX> Specified objective sense:

CPLEX> AVERAGE_ELECTRICITY_PRICE : Size=1
    Key  : Value
    None : 0.2605976401867579
None
tau : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :  None :   0.0 :  None : False : False :  Reals
None
Emissions intensity baseline: 1.0750000000000002 tCO2/MWh

Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 12.10.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2019.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile '/home/compo/Desktop/git/research/phd-thesis/repos/tps-parameterisation/src/2_parameter_selector/kkt-approach/tmp053xiryr.cplex.log' open.
CPLEX> New value for absolute mixed integer optimality gap tolerance: 1e-06
CPLEX> New value for emphasis for MIP optimization: 1
CPLEX> Specified objective sense:

CPLEX> AVERAGE_ELECTRICITY_PRICE : Size=1
    Key  : Value
    None : 0.2605976401867579
None
tau : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :  None :   0.0 :  None : False : False :  Reals
None
Emissions intensity baseline: 1.0650000000000002 tCO2/MWh

Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 12.10.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2019.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile '/home/compo/Desktop/git/research/phd-thesis/repos/tps-parameterisation/src/2_parameter_selector/kkt-approach/tmp8cpjkw7m.cplex.log' open.
CPLEX> New value for absolute mixed integer optimality gap tolerance: 1e-06
CPLEX> New value for emphasis for MIP optimization: 1
CPLEX> Specified objective sense:

...

  21014   861    infeasible                            0.0000  2417682         
  21315   850        0.0000    90                      0.0000  2465629         
  21617   892    infeasible                            0.0000  2535575         
  21933   891        0.0000   131                      0.0000  2619529         
  22217   889        0.0000   121                      0.0000  2656205         
  22499   866        0.0000   119                      0.0000  2719364         
  22767   845        0.0000   174                      0.0000  2767222         
  23070   879        0.0000    87                      0.0000  2837442         
  23376   845        0.0000   114                      0.0000  2888461         
Elapsed time = 242.28 sec. (214820.02 ticks, tree = 0.66 MB, solutions = 0)
  23670   891        0.0000    99                      0.0000  2926535         
  23933   884    infeasible                            0.0000  2991547         
  24236   884        0.0000   143           

CPLEX> AVERAGE_ELECTRICITY_PRICE : Size=1
    Key  : Value
    None : 0.3925366745135814
None
tau : Size=1, Index=None
    Key  : Lower : Value              : Upper : Fixed : Stale : Domain
    None :  None : 1.1749853228775315 :  None : False : False :  Reals
None
Emissions intensity baseline: 0.9 tCO2/MWh

Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 12.10.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2019.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile '/home/compo/Desktop/git/research/phd-thesis/repos/tps-parameterisation/src/2_parameter_selector/kkt-approach/tmpdv6ms_t8.cplex.log' open.
CPLEX> New value for absolute mixed integer optimality gap tolerance: 1e-06
CPLEX> New value for emphasis for MIP optimization: 1
CPLEX> Specified objec

  13450  1678        0.0000   102                      0.0000   663356         
  14088  1563        0.0000   150                      0.0000   693017         
  14561  1566        0.0000    65                      0.0000   713561         
  14845  1553        0.0000   101                      0.0000   741088         
  15439  1566        0.0000   111                      0.0000   761811         
  15993  1598    infeasible                            0.0000   777141         
  16527  1607        0.0000    73                      0.0000   807781         
  16743  1622        0.0000   154                      0.0000   827134         
Elapsed time = 56.36 sec. (46201.04 ticks, tree = 1.80 MB, solutions = 0)
  17126  1571    infeasible                            0.0000   850106         
  17625  1587        0.0000    56                      0.0000   866534         
  18116  1559        0.0000   121                      0.0000   884047         
  18500  1517        0.0000    83             

  62042   626        0.0000    52                      0.0000  3102270         
  62748   790    infeasible                            0.0000  3158750         
  63266   963    infeasible                            0.0000  3202207         
  63890  1202    infeasible                            0.0000  3249519         
Elapsed time = 211.51 sec. (174526.89 ticks, tree = 1.85 MB, solutions = 0)
  64445  1347    infeasible                            0.0000  3300061         
  64953  1422        0.0000   109                      0.0000  3356632         
  65464  1405        0.0000   100                      0.0000  3418069         
  65971  1467        0.0000    57                      0.0000  3475608         
  66619  1526        0.0000    58                      0.0000  3531510         
  67237  1601    infeasible                            0.0000  3589639         
  67747  1615        0.0000    55                      0.0000  3649463         
  68388  1664        0.0000    63           

 108004   610    infeasible                            0.0000  8976036         
 108667   634        0.0000    38                      0.0000  9018498         
 109324   653    infeasible                            0.0000  9070615         
 109892   623        0.0000    22                      0.0000  9178481         
 110258   585        0.0000   149                      0.0000  9230274         
 110716   533    infeasible                            0.0000  9318751         
 111375   550        0.0000    79                      0.0000  9369043         
 112126   564    infeasible                            0.0000  9460544         
 112769   557        0.0000    30                      0.0000  9484839         
 113296   565        0.0000   110                      0.0000  9550472         
Elapsed time = 659.73 sec. (556800.36 ticks, tree = 0.49 MB, solutions = 0)
 113847   554        0.0000   130                      0.0000  9630518         
 114302   534        0.0000    26           

Identify baseline that targets wholesale electricity price

In [ ]:
def run_weighted_rrn_price_target_scenarios():
    "Run model that targets weighted RRN electricity prices"
    
    # Run BAU model (very high baseline)
    model_bau = create_model(use_pu=True, variable_baseline=False, objective_type='feasibility')
    model_bau.PHI = 1.5
       
    # BAU model results
    print('Solving BAU scenario')
    res_bau = opt.solve(model_bau, keepfiles=False, tee=True, warmstart=True)    
    model_bau.solutions.store_to(res_bau)
    bau_weighted_rnn_price = model_bau.WEIGHTED_RRN_PRICE.expr()
    print('BAU weighted RNN price: {0}'.format(bau_weighted_rnn_price))

    # Instantiate model object
    model = create_model(use_pu=True, variable_baseline=True, objective_type='weighted_rrn_price_target')
    opt.options['mip tolerances absmipgap'] = 1e-3

    # Weighted RNN price target as a multiple of BAU weighted RNN price
    for price_multiplier in [1.1, 1.2, 1.3, 1.4, 0.8]:
        # Update price target
        model.WEIGHTED_RRN_PRICE_TARGET = price_multiplier * bau_weighted_rnn_price
        print('Target price input: {}'.format(price_multiplier * bau_weighted_rnn_price))

        # Model results
        res = opt.solve(model, keepfiles=False, tee=True, warmstart=True)
        model.solutions.store_to(res)

        # Place results in DataFrame
        try:
            df = pd.DataFrame(res['Solution'][0])
            price_target_results = {'WEIGHTED_RRN_PRICE_TARGET': model.WEIGHTED_RRN_PRICE_TARGET.value,
                                    'WEIGHTED_RRN_PRICE_TARGET_BAU_MULTIPLE': price_multiplier,
                                    'results': df,
                                    'PHI_DISCRETE': model.PHI_DISCRETE.expr()}
        except:
            price_target_results = {'WEIGHTED_RRN_PRICE_TARGET': model.WEIGHTED_RRN_PRICE_TARGET.value,
                                    'results': 'infeasible'}
            print('Weighted RNN price target {0} is infeasible'.format(model.WEIGHTED_RRN_PRICE_TARGET.value))   

        # Try to print results
        try:
            print(model.WEIGHTED_RRN_PRICE.display())
        except:
            pass

        try:
            print(model.PHI_DISCRETE.display())
        except:
            pass

        print(model.tau.display())

        # Save results
        filename = 'weighted_rrn_price_target_{0:.3f}.pickle'.format(price_multiplier)
        with open(os.path.join(paths.output_dir, filename), 'wb') as f:
            pickle.dump(price_target_results, f)
           
run_weighted_rrn_price_target_scenarios()

Flow directions conform with regional flow limit directions: True
Flow directions conform with regional flow limit directions: True
Flow directions conform with regional flow limit directions: True
Solving BAU scenario

Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 12.10.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2019.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile '/home/compo/Desktop/git/research/phd-thesis/repos/tps-parameterisation/src/2_parameter_selector/kkt-approach/tmptls62dyx.cplex.log' open.
CPLEX> New value for absolute mixed integer optimality gap tolerance: 1e-06
CPLEX> New value for emphasis for MIP optimization: 1
CPLEX> Specified objective sense: MINIMIZE
Selected objective  name:  x16566
Selected RHS        name:  RHS
Sele

CPLEX> BAU weighted RNN price: 0.25952887468793423
Flow directions conform with regional flow limit directions: True
Flow directions conform with regional flow limit directions: True
Flow directions conform with regional flow limit directions: True
Target price input: 0.28548176215672766

Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 12.10.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2019.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile '/home/compo/Desktop/git/research/phd-thesis/repos/tps-parameterisation/src/2_parameter_selector/kkt-approach/tmp6eq2_gcc.cplex.log' open.
CPLEX> New value for absolute mixed integer optimality gap tolerance: 0.001
CPLEX> New value for emphasis for MIP optimization: 1
CPLEX> Specified objective sense: MINIMIZE

   1314   802        0.0000   221                      0.0000   316075         
   1361   831        0.0000   122                      0.0000   320266         
   1409   891        0.0000   126                      0.0000   335864         
   1450   878    infeasible                            0.0000   330155         
   1483   912    infeasible                            0.0000   348794         
   1519   940        0.0000   221                      0.0000   356548         
   1584   957        0.0000   159                      0.0000   359134         
   1669  1036        0.0000   167                      0.0000   383566         
   1714  1057        0.0000   115                      0.0000   386407         
Elapsed time = 162.17 sec. (132164.66 ticks, tree = 77.00 MB, solutions = 0)
   1739  1090        0.0000   224                      0.0000   394009         
   1745  1091        0.0000   209                      0.0000   401828         
   1752  1094        0.0000   161          

  13663     2        0.0000   128                      0.0000  2058344         
  13914   118        0.0000   140                      0.0000  2091259         
  14282   285        0.0000   109                      0.0000  2131575         
  14755   467        0.0000    52                      0.0000  2174402         
  15277   563    infeasible                            0.0000  2215599         
Elapsed time = 374.86 sec. (304803.85 ticks, tree = 1.73 MB, solutions = 0)
  15518   602        0.0000   196                      0.0000  2248903         
  15934   695        0.0000    66                      0.0000  2286394         
  16172   713        0.0000   183                      0.0000  2316113         
  16518   714        0.0000   100                      0.0000  2357143         
  16738   742        0.0000   179                      0.0000  2386402         
  16899   808    infeasible                            0.0000  2434940         
  17063   800        0.0000   165           

  32593  1483       -0.0000    41                      0.0000  5649242         
Elapsed time = 743.54 sec. (621682.79 ticks, tree = 3.66 MB, solutions = 0)
  32872  1491       -0.0000    47                      0.0000  5699359         
  33242  1486    infeasible                            0.0000  5773705         
  33512  1503        0.0000    95                      0.0000  5809989         
  33775  1506        0.0000    68                      0.0000  5876741         
  34101  1502        0.0000   152                      0.0000  5930432         
  34373  1499    infeasible                            0.0000  5908333         
  34569  1492    infeasible                            0.0000  5980688         
  34795  1493    infeasible                            0.0000  6049706         
  35080  1498    infeasible                            0.0000  6112450         
  35340  1517        0.0000   173                      0.0000  6153134         
Elapsed time = 789.03 sec. (660007.01 ticks,

...

 226815  8410    infeasible              0.0011        0.0000 67383249  100.00%
 227280  8375    infeasible              0.0011        0.0000 67493070  100.00%
 227647  8319    infeasible              0.0011        0.0000 67791252  100.00%
 228261  8323    infeasible              0.0011        0.0000 67964113  100.00%
 228907  8359    infeasible              0.0011        0.0000 68007866  100.00%
 229712  8337        0.0000    63        0.0011        0.0000 68222384  100.00%
 230416  8333        0.0000    87        0.0011        0.0000 68473249  100.00%
 231200  8374    infeasible              0.0011        0.0000 68829457  100.00%
Elapsed time = 6837.31 sec. (5136751.50 ticks, tree = 24.75 MB, solutions = 6)
 232165  8366        0.0000    67        0.0011        0.0000 69021185  100.00%
 233129  8359    infeasible              0.0011        0.0000 69286390  100.00%
 233450  8323        0.0000   171        0.0011        0.0000 69334649  100.00%
 233681  8308        0.0000    90        

 268475  8743    infeasible              0.0011        0.0000 86203678  100.00%
 268740  8759        0.0000   174        0.0011        0.0000 86623362  100.00%
 269068  8744    infeasible              0.0011        0.0000 86694404  100.00%
 269496  8782    infeasible              0.0011        0.0000 86803252  100.00%
Elapsed time = 9440.60 sec. (6516307.34 ticks, tree = 25.88 MB, solutions = 6)
 270281  8805        0.0000    35        0.0011        0.0000 87207028  100.00%
 271079  8826    infeasible              0.0011        0.0000 87263360  100.00%
 271453  8871        0.0001    20        0.0011        0.0000 87432164  100.00%
 271860  8992    infeasible              0.0011        0.0000 87749336  100.00%
 272850  9050    infeasible              0.0011        0.0000 87872386  100.00%
 273702  9152    infeasible              0.0011        0.0000 88255596  100.00%
 274441  9196        0.0000   112        0.0011        0.0000 88414562  100.00%
 275052  9230        0.0006    12        

 309926  9329        0.0000    83        0.0011        0.0000 1.06e+08  100.00%
 310267  9326    infeasible              0.0011        0.0000 1.06e+08  100.00%
 310555  9291        0.0000   162        0.0011        0.0000 1.06e+08  100.00%
 310743  9286        0.0000   230        0.0011        0.0000 1.06e+08  100.00%
 310892  9273        0.0000    88        0.0011        0.0000 1.07e+08  100.00%
 310989  9265        0.0000   178        0.0011        0.0000 1.07e+08  100.00%
 311218  9313        0.0000   142        0.0011        0.0000 1.07e+08  100.00%
 311412  9300        0.0000   163        0.0011        0.0000 1.07e+08  100.00%
 311651  9355    infeasible              0.0011        0.0000 1.07e+08  100.00%
 311834  9344        cutoff              0.0011        0.0000 1.07e+08  100.00%
Elapsed time = 11573.66 sec. (8050952.84 ticks, tree = 27.55 MB, solutions = 6)
 311963  9326        0.0000   241        0.0011        0.0000 1.08e+08  100.00%
 312127  9314        cutoff             

 332886  8636    infeasible              0.0011        0.0000 1.22e+08  100.00%
 333231  8635        0.0000   146        0.0011        0.0000 1.22e+08  100.00%
 333533  8588    infeasible              0.0011        0.0000 1.23e+08  100.00%
 333925  8558        0.0000    78        0.0011        0.0000 1.23e+08  100.00%
 334368  8550        0.0000    83        0.0011        0.0000 1.23e+08  100.00%
 334815  8474        cutoff              0.0011        0.0000 1.23e+08  100.00%
Elapsed time = 14057.92 sec. (9433606.30 ticks, tree = 24.77 MB, solutions = 6)
 335462  8481        0.0000    91        0.0011        0.0000 1.23e+08  100.00%
 335792  8457    infeasible              0.0011        0.0000 1.23e+08  100.00%
 336095  8432        0.0000    79        0.0011        0.0000 1.24e+08  100.00%
 336388  8368        0.0000   181        0.0011        0.0000 1.24e+08  100.00%
 336649  8405        0.0000   193        0.0011        0.0000 1.24e+08  100.00%
 336867  8315    infeasible             

 367862  6880    infeasible              0.0011        0.0000 1.40e+08  100.00%
 369048  6906        0.0000    23        0.0011        0.0000 1.40e+08  100.00%
Elapsed time = 16466.66 sec. (10814819.61 ticks, tree = 19.74 MB, solutions = 6)
 369885  6839        0.0000    61        0.0011        0.0000 1.40e+08  100.00%
 370749  6893    infeasible              0.0011        0.0000 1.41e+08  100.00%
 371576  6921    infeasible              0.0011        0.0000 1.41e+08  100.00%
 371944  6919    infeasible              0.0011        0.0000 1.41e+08  100.00%
 372583  6926        0.0000    43        0.0011        0.0000 1.41e+08  100.00%
 373261  6928    infeasible              0.0011        0.0000 1.41e+08  100.00%
 374702  6914        0.0000    53        0.0011        0.0000 1.42e+08  100.00%
 375356  6915        0.0000    49        0.0011        0.0000 1.42e+08  100.00%
 376081  6926    infeasible              0.0011        0.0000 1.42e+08  100.00%
 377200  6877    infeasible            

Identify baseline that targets equilibrium permit price

In [ ]:
def run_permit_price_target_scenarios():
    "Run model for different permit price target scenarios"

    # Instantiate model object
    model = create_model(use_pu=True, variable_baseline=True, objective_type='permit_price_target')
    opt.options['mip tolerances absmipgap'] = 1e-3

    for permit_price in [25/100, 50/100, 75/100, 100/100]:
        # Set permit price target
        model.PERMIT_PRICE_TARGET = permit_price

        # Model results
        res = opt.solve(model, keepfiles=False, tee=True, warmstart=True)
        model.solutions.store_to(res)

        # Place results in DataFrame
        try:
            df = pd.DataFrame(res['Solution'][0])
            permit_price_target_results = {'PERMIT_PRICE_TARGET': permit_price,
                                           'results': df,
                                           'PHI_DISCRETE': model.PHI_DISCRETE.expr()}
        except:
            permit_price_target_results = {'PERMIT_PRICE_TARGET': permit_price,
                                           'results': 'infeasible'}
            print('Permit price target {0} is infeasible'.format(permit_price))

        # Try to print results
        try:
            print(model.PHI_DISCRETE.display())
        except:
            pass

        print(model.tau.display())

        # Save results
        filename = 'permit_price_target_{0:.3f}.pickle'.format(permit_price)
        with open(os.path.join(paths.output_dir, filename), 'wb') as f:
            pickle.dump(permit_price_target_results, f)
            
run_permit_price_target_scenarios()